In [3]:
import pandas as pd
import pickle as pkl
from functions import *
from process_text import *
import os

In [8]:
#load data and save answers
test_tweets = pd.read_csv('hatespeech-test.csv', index_col = 0)
train_tweets = pd.read_csv('hatespeech-train.csv', index_col = 0)

all_separated = [test_tweets, train_tweets]
all_separated = [dic["tweet"].values.tolist() for dic in all_separated]
answers = [np.identity(3)[x['class'].values.tolist()] for x in [test_tweets, train_tweets]]
answers2 = [x['class'].values.tolist() for x in [test_tweets, train_tweets]]
save_pkl('cleaned_corpus/answer_data.pkl', answers)
save_pkl('cleaned_corpus/org_answer_data.pkl', answers2)

In [4]:
train_tweets['class'].value_counts()

1    4500
2    3600
0    1260
Name: class, dtype: int64

In [5]:
test_tweets['class'].value_counts()

1    500
2    400
0    140
Name: class, dtype: int64

In [14]:
length = len(all_separated[0])+ len(all_separated[1])
counter = 0
tokenized = []
client = nlp_client()

for sets in all_separated:
    data = []
    for tweet in sets:
        text = filter_tw(tweet)       
        text = client.annotate(text)
        text = additional_fix(text, client)
        data.append(text)
        counter += 1
        if counter %1040 == 0: print(str(counter)+'/'+ str(length))
        
    tokenized.append(data)

save_pkl('cleaned_corpus/base_cleaned.pkl', tokenized)
    
    

1040/10400
2080/10400
3120/10400
4160/10400
5200/10400
6240/10400
7280/10400
8320/10400
9360/10400
10400/10400


In [15]:
vectorized = gen_model_input(tokenized)

In [16]:
save_pkl('cleaned_corpus/base_model_input.pkl', vectorized)

In [1]:
def read_txt(dest):
    with open(dest, 'r', encoding = 'utf-8') as f:
        return f.read()

In [2]:
fr = read_txt('trans/transed/fr_en.txt')
gr = read_txt('trans/transed/gr_en.txt')
du = read_txt('trans/transed/du_en.txt')

fr_n = read_txt('trans/transed/fr_en_n.txt')

In [4]:
trans_hate = []
for lang in [fr, gr, du]:
    lang = [x for x in lang.split('\n') if len(x) != 0]
    trans_hate.extend(lang)
trans_neither = fr_n.split('\n')

In [5]:
random.shuffle(trans_hate)
add_hate = trans_hate[:4500-1260]

In [6]:
length = len(add_hate)
counter = 0
client = nlp_client()
add_data = []
for sets in [add_hate, trans_neither]:
    for tweet in sets:
        text = client.annotate(tweet)
        text = additional_fix(text, client)
        text = [re.sub('<answer>', '<reply>', word.lower()) for word in text]
        add_data.append(text)
        counter += 1
        if counter %500 == 0: print(str(counter)+'/'+ str(length))    

500/3240
1000/3240
1500/3240
2000/3240
2500/3240
3000/3240
3500/3240
4000/3240


In [9]:
data = load_pkl('cleaned_corpus/base_cleaned.pkl')
test, train = data
test_a, train_a = answers
test_a2, train_a2 = answers2

In [12]:
from copy import deepcopy
train_X = deepcopy(train)
train_X.extend(add_data)

In [13]:
new_corpus = (test, train_X)
save_pkl('cleaned_corpus/augmented_corpus.pkl', new_corpus)

In [14]:
m_input = gen_model_input(new_corpus)
save_pkl('cleaned_corpus/augmented_model_input.pkl', m_input)

In [15]:
add_hate_ans = np.array([[1.0, 0.0, 0.0] for _ in range(len(add_hate))])
add_nei_ans = np.array([[0.0, 0.0, 1.0] for _ in range(len(trans_neither))])

In [16]:
new_oha = np.append(train_a, add_hate_ans, axis = 0)
new_oha = np.append(new_oha, add_nei_ans, axis = 0)

In [17]:
oha = (test_a, new_oha)

In [18]:
save_pkl('cleaned_corpus/augmented_ans.pkl', oha)

In [19]:
new_org_a = np.argmax(new_oha, axis = 1)
new_ans = (test_a2, new_org_a)

In [20]:
save_pkl('cleaned_corpus/augmented_org_ans.pkl', new_ans)